# ROS Control MicroCourse

## Chapter 3: Creating a Controller

<p style="background:green;color:white;">SUMMARY</p>

Estimated time to completion: <b>1 hour</b><br><br>
This unit will explain how to create a basic controller for your robot.

<p style="background:green;color:white;">END OF SUMMARY</p>

So, up until this point, you've seen how you can connect your simulated robot to the ros_control packages to be able to control its joints. And you are using the default controllers that the ros_control packages provide, which is the most common practice. In some cases, it may be interesting to create a custom ROS controller that sends specific commands to your joints, don't you think?

And that's exactly what you are going to learn to do in this unit! So, in order to see how you can achieve this, follow the next exercise.

<p style="background:#EE9023;color:white;">Exercise 3.1</p>
### Creating the package
<br>
Create a new package named my_controller with the following dependencies: <i><b>roscpp</b></i>, <i><b>pluginlib</b></i>, <i><b>controller_interface</b></i>, and <i><b>hardware_interface</b></i>. To do that, just execute the following command:
<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

### Creating the Source code

Inside the src folder that was created in your new package, create a file named <i><b>my_controller.cpp</b></i>. Copy the following code into the file:

Are you a little bit confused right now? Don't worry! Let's explain the code:

In [ ]:
#include <controller_interface/controller.h>
#include <hardware_interface/joint_command_interface.h>
#include <pluginlib/class_list_macros.h>

namespace controller_ns{

Here, you are just including some files from other packages that are needed in order to create the controller, and declaring a namespace for this class (namespaces provide a method to prevent name conflicts in large projects).

In [ ]:
class PositionController : public controller_interface::Controller<hardware_interface::EffortJointInterface>

Here you are just declaring the class, which will be inherited from <i><b>hardware_interface::EffortJointInterface</b></i>. This means that this controller will be able to control only joints that use an effort interface.

In [ ]:
bool init(hardware_interface::EffortJointInterface* hw, ros::NodeHandle &n)
{
  std::string my_joint;
  if (!n.getParam("joint", my_joint)){
    ROS_ERROR("Could not find joint name");
    return false;
  }
  joint_ = hw->getHandle(my_joint);  // throws on failure
  return true;
}

Here, you are creating the init() function, which will be called when your controller is loaded by the controller manager. Inside this function, you will get the name of the joint that you will control from the Parameter Server first (so from the YAML file, which you will modify later), and then you will get the joint object to use in the realtime loop.

In [ ]:
void update(const ros::Time& time, const ros::Duration& period)
{
double error = setpoint_ - joint_.getPosition();
joint_.setCommand(error*gain_);
}

Here you are defining the command that you are going to send to your joint. In this case, it's a product between an <i><b>error</b></i> variable and a <i><b>gain_</b></i> variable. The error variable is defined as the difference between the current position (<i><b>joint_.getPosition()</b></i>) of the joint and the goal position (<i><b>setpoint_</b></i>) of the joint.

In [ ]:
void starting(const ros::Time& time) { }
void stopping(const ros::Time& time) { }

Here, you are starting and stopping the controller.

In [ ]:
private:
hardware_interface::JointHandle joint_;
static const double gain_ = 2.25;
static const double setpoint_ = 1.00;

Here, you are just defining a variable that you'll use through the class code.

In [ ]:
PLUGINLIB_EXPORT_CLASS(controller_ns::PositionController, controller_interface::ControllerBase);

Here, you are calling the special macro plugin <i><b>PLUGINLIB_EXPORT_CLASS</b></i> in order to allow this class to be dynamically loaded.

### Creating a plugin description file

Inside your package, create a new file named <i><b>controller_plugins.xml</b></i>. Copy the following code into it:

This file is a description of the controller plugin. It indicates the path where the library will be placed when we compile the controller, as well as the name, type, and class of the controller.

### Updating the package.xml file

Go to the <i><b>package.xml</b></i> file of your package. Inside the <i><b>&lt;export&gt;</b></i> tag, place the following line:

This line indicates that our package will provide a plugin. This is very important because if we don't add this line, the controller manager won't be able to find our new controller and load it.

### Updating the CMakeLists.txt file

Go to the <i><b>CMakeLists.txt</b></i> file of your package. Find the <i><b>add_library()</b></i> and <i><b>target_link_libraries()</b></i> functions, and replace them with these:

### Build the controller

Go to the <i><b>catkin_ws</b></i> directory and compile your package.

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

Now, check if your controller has been properly registered by using the following command:

If everything has gone well, you should visualize your plugin in the controllers list.

<img src="img/my_controller.png" width="1000" />

### Write the configuration file

Inside the package, create a new folder named <i><b>config</b></i>, with a file inside named <i><b>my_controller.yaml</b></i>, and copy the contents of the YAML file you created in the previous chapter into this file. Finally, modify the joint 1 configuration in order for it to be controlled by your new controller.

<p style="color:red;"><b>SOLUTION: DO NOT LOOK unless you are really stuck.</b></p>

### Create a launch file

Inside the package, create a new folder named <i><b>launch</b></i>, with a launch file inside named <i><b>my_controller.launch</b></i>, that launches the control system of the robot.

<p style="background:#AE0202;color:white;">Expected Result for Exercise 3.1</p>

If you launch your controller with the robot at its start position, you should get something like this:

<img src="img/controller_move.png" width="300" />

Keep in mind that depending on how the robot's joints are placed after you moved them in the previous chapter, this could change.

<p style="background:#EE9023;color:white;">End of Exercise 3.1</p>

And that's it!! You have finally completed the course. Well, almost... Now, if you want to test all that you've learned in a course project, go to the next unit!